# Video Frame Streaming Server

Server WebSocket để stream từng frame của video cho ứng dụng Flutter.

## Features:
- Đọc video frame by frame
- Chuyển đổi frame thành base64 
- Stream qua WebSocket port 8081
- Hỗ trợ multiple clients
- Tự động loop video

## Video file: 
`C:\Users\sonng\Code\Test\2.jpg 2025-08-11 16-32-53.mp4`

In [4]:
import cv2
import asyncio
import websockets
import json
import base64
import threading
import time
import os
from datetime import datetime

class VideoFrameServer:
    def __init__(self, video_path, host="localhost", port=8081):
        self.video_path = video_path
        self.host = host
        self.port = port
        self.clients = set()
        self.is_running = False
        self.current_frame = None
        self.frame_count = 0
        self.fps = 30  # Default FPS
        
        # Kiểm tra video file
        if not os.path.exists(video_path):
            raise FileNotFoundError(f"Video file không tồn tại: {video_path}")
            
        # Lấy thông tin video
        cap = cv2.VideoCapture(video_path)
        self.total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        self.fps = cap.get(cv2.CAP_PROP_FPS) or 30
        self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        cap.release()
        
        print(f"📹 Video Info:")
        print(f"   📁 File: {video_path}")
        print(f"   📏 Resolution: {self.width}x{self.height}")
        print(f"   🎬 Total Frames: {self.total_frames}")
        print(f"   ⚡ FPS: {self.fps}")
        print(f"   ⏱️ Duration: {self.total_frames/self.fps:.2f}s")
    
    async def register_client(self, websocket, path):
        """Đăng ký client mới"""
        self.clients.add(websocket)
        client_info = f"{websocket.remote_address[0]}:{websocket.remote_address[1]}"
        print(f"✅ Client connected: {client_info} (Total: {len(self.clients)})")
        
        try:
            # Gửi thông tin video cho client mới
            video_info = {
                "type": "video_info",
                "width": self.width,
                "height": self.height,
                "fps": self.fps,
                "total_frames": self.total_frames,
                "timestamp": datetime.now().isoformat()
            }
            await websocket.send(json.dumps(video_info))
            
            # Giữ connection alive
            await websocket.wait_closed()
            
        except websockets.exceptions.ConnectionClosed:
            pass
        except Exception as e:
            print(f"❌ Client error: {e}")
        finally:
            self.clients.remove(websocket)
            print(f"❌ Client disconnected: {client_info} (Remaining: {len(self.clients)})")
    
    async def broadcast_frame(self, frame_data):
        """Gửi frame tới tất cả clients"""
        if self.clients:
            # Tạo message với frame data
            message = {
                "type": "video_frame",
                "frame_id": self.frame_count,
                "timestamp": datetime.now().isoformat(),
                "image_data": frame_data,
                "width": self.width,
                "height": self.height
            }
            
            # Gửi tới tất cả clients
            disconnected_clients = []
            for client in self.clients.copy():
                try:
                    await client.send(json.dumps(message))
                except websockets.exceptions.ConnectionClosed:
                    disconnected_clients.append(client)
                except Exception as e:
                    print(f"❌ Broadcast error: {e}")
                    disconnected_clients.append(client)
            
            # Remove disconnected clients
            for client in disconnected_clients:
                self.clients.discard(client)
    
    def frame_processor(self):
        """Xử lý video frames trong thread riêng"""
        cap = cv2.VideoCapture(self.video_path)
        frame_delay = 1.0 / self.fps  # Delay giữa các frame
        
        print(f"🚀 Frame processor started - FPS: {self.fps}")
        
        while self.is_running:
            ret, frame = cap.read()
            
            if not ret:
                # Restart video từ đầu
                print("🔄 Video ended, restarting...")
                cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                self.frame_count = 0
                continue
            
            # Resize frame nếu cần (để tối ưu bandwidth)
            # frame = cv2.resize(frame, (640, 480))  # Uncomment nếu muốn resize
            
            # Chuyển frame thành base64
            _, buffer = cv2.imencode('.jpg', frame, [cv2.IMWRITE_JPEG_QUALITY, 80])
            frame_base64 = base64.b64encode(buffer).decode('utf-8')
            
            self.current_frame = frame_base64
            self.frame_count += 1
            
            # Broadcast tới clients
            if self.clients:
                asyncio.run_coroutine_threadsafe(
                    self.broadcast_frame(frame_base64), 
                    self.loop
                )
            
            # Thống kê
            if self.frame_count % 100 == 0:
                print(f"📊 Processed {self.frame_count} frames, Active clients: {len(self.clients)}")
            
            # Delay để maintain FPS
            time.sleep(frame_delay)
        
        cap.release()
        print("🛑 Frame processor stopped")
    
    async def start_server(self):
        """Khởi động WebSocket server"""
        self.is_running = True
        self.loop = asyncio.get_event_loop()
        
        # Start frame processor thread
        frame_thread = threading.Thread(target=self.frame_processor)
        frame_thread.daemon = True
        frame_thread.start()
        
        # Start WebSocket server
        print(f"🚀 Starting WebSocket server on {self.host}:{self.port}")
        
        async with websockets.serve(self.register_client, self.host, self.port):
            print(f"✅ Server running on ws://{self.host}:{self.port}")
            print("📱 Flutter app can connect to this URL")
            print("🛑 Press Ctrl+C to stop")
            
            try:
                await asyncio.Future()  # Run forever
            except KeyboardInterrupt:
                print("🛑 Server stopping...")
                self.is_running = False

# Khởi tạo server
video_file = r"C:\Users\sonng\Code\Test\2.jpg 2025-08-11 16-32-53.mp4"
server = VideoFrameServer(video_file)

print("🎬 Video Frame Streaming Server Ready!")
print("=" * 50)

📹 Video Info:
   📁 File: C:\Users\sonng\Code\Test\2.jpg 2025-08-11 16-32-53.mp4
   📏 Resolution: 1920x1080
   🎬 Total Frames: 234
   ⚡ FPS: 16.634992251221558
   ⏱️ Duration: 14.07s
🎬 Video Frame Streaming Server Ready!


In [ ]:
# Chạy server (chạy cell này để start server)
# Server sẽ chạy cho đến khi restart kernel

try:
    await server.start_server()
except KeyboardInterrupt:
    print("🛑 Server stopped by user")
except Exception as e:
    print(f"❌ Server error: {e}")

🚀 Starting WebSocket server on localhost:8081
✅ Server running on ws://localhost:8081
📱 Flutter app can connect to this URL
🛑 Press Ctrl+C to stop
🚀 Frame processor started - FPS: 16.634992251221558
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...
📊 Processe

connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'
connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'


📊 Processed 100 frames, Active clients: 0
📊 Processed 200 frames, Active clients: 0
🔄 Video ended, restarting...


connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'
connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'


📊 Processed 100 frames, Active clients: 0


...

connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'
connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'
connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1

📊 Processed 200 frames, Active clients: 0


connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'


🔄 Video ended, restarting...


connection handler failed
Traceback (most recent call last):
  File "c:\Users\sonng\miniconda3\envs\autovrs\Lib\site-packages\websockets\asyncio\server.py", line 376, in conn_handler
    await self.handler(connection)
          ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: VideoFrameServer.register_client() missing 1 required positional argument: 'path'


In [3]:
# Test client để kiểm tra server (chạy trong cell riêng)
import asyncio
import websockets
import json
import base64
from PIL import Image
import io
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

class VideoStreamClient:
    def __init__(self, uri="ws://localhost:8081"):
        self.uri = uri
        self.frame_count = 0
        
    async def connect_and_display(self):
        try:
            print(f"🔗 Connecting to {self.uri}...")
            
            async with websockets.connect(self.uri) as websocket:
                print("✅ Connected to video stream!")
                
                async for message in websocket:
                    try:
                        data = json.loads(message)
                        
                        if data["type"] == "video_info":
                            print(f"📹 Video Info Received:")
                            print(f"   📏 Resolution: {data['width']}x{data['height']}")
                            print(f"   ⚡ FPS: {data['fps']}")
                            print(f"   🎬 Total Frames: {data['total_frames']}")
                            print("-" * 50)
                            
                        elif data["type"] == "video_frame":
                            self.frame_count += 1
                            
                            # Decode base64 image
                            image_data = base64.b64decode(data["image_data"])
                            image = Image.open(io.BytesIO(image_data))
                            
                            # Display frame (hiển thị mỗi 30 frame để tránh spam)
                            if self.frame_count % 30 == 0:
                                clear_output(wait=True)
                                
                                plt.figure(figsize=(10, 6))
                                plt.imshow(image)
                                plt.title(f"Video Frame #{data['frame_id']} - Client Frame #{self.frame_count}")
                                plt.axis('off')
                                plt.show()
                                
                                print(f"📺 Frame #{data['frame_id']} displayed")
                                print(f"⏰ Timestamp: {data['timestamp']}")
                                print(f"📊 Total received: {self.frame_count} frames")
                        
                    except json.JSONDecodeError:
                        print("❌ Invalid JSON received")
                    except Exception as e:
                        print(f"❌ Frame processing error: {e}")
                        
        except websockets.exceptions.ConnectionClosed:
            print("🔌 Connection closed")
        except ConnectionRefusedError:
            print("❌ Cannot connect - Server not running?")
        except Exception as e:
            print(f"❌ Client error: {e}")

# Tạo test client
print("🧪 Video Stream Test Client")
print("=" * 40)
client = VideoStreamClient()

# Uncomment dòng dưới để test (chạy sau khi server đã start)
# await client.connect_and_display()

ModuleNotFoundError: No module named 'matplotlib'

# Flutter Client Code

## Để kết nối từ Flutter app, sử dụng code sau:

### 1. Thêm dependencies trong `pubspec.yaml`:
```yaml
dependencies:
  web_socket_channel: ^2.4.0
  flutter: 
    sdk: flutter
```

### 2. Flutter Widget Code:

```dart
import 'package:flutter/material.dart';
import 'package:web_socket_channel/web_socket_channel.dart';
import 'dart:convert';
import 'dart:typed_data';

class VideoStreamWidget extends StatefulWidget {
  @override
  _VideoStreamWidgetState createState() => _VideoStreamWidgetState();
}

class _VideoStreamWidgetState extends State<VideoStreamWidget> {
  WebSocketChannel? _channel;
  Uint8List? _currentFrame;
  int _frameCount = 0;
  bool _isConnected = false;

  @override
  void initState() {
    super.initState();
    _connectToServer();
  }

  void _connectToServer() {
    try {
      _channel = WebSocketChannel.connect(
        Uri.parse('ws://YOUR_SERVER_IP:8081'), // Thay YOUR_SERVER_IP bằng IP thực
      );
      
      _channel!.stream.listen(
        (data) {
          final message = json.decode(data);
          
          if (message['type'] == 'video_info') {
            print('Video Info: ${message['width']}x${message['height']}');
            setState(() {
              _isConnected = true;
            });
          } else if (message['type'] == 'video_frame') {
            final imageData = base64Decode(message['image_data']);
            setState(() {
              _currentFrame = imageData;
              _frameCount++;
            });
          }
        },
        onError: (error) {
          print('WebSocket Error: $error');
          setState(() {
            _isConnected = false;
          });
        },
        onDone: () {
          print('WebSocket Connection Closed');
          setState(() {
            _isConnected = false;
          });
        },
      );
    } catch (e) {
      print('Connection Error: $e');
    }
  }

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Video Stream'),
        backgroundColor: _isConnected ? Colors.green : Colors.red,
      ),
      body: Column(
        children: [
          Container(
            padding: EdgeInsets.all(16),
            child: Row(
              mainAxisAlignment: MainAxisAlignment.spaceAround,
              children: [
                Text('Status: ${_isConnected ? "Connected" : "Disconnected"}'),
                Text('Frames: $_frameCount'),
              ],
            ),
          ),
          Expanded(
            child: Center(
              child: _currentFrame != null
                  ? Image.memory(
                      _currentFrame!,
                      fit: BoxFit.contain,
                    )
                  : _isConnected
                      ? CircularProgressIndicator()
                      : Text('Connecting to video stream...'),
            ),
          ),
        ],
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          if (_isConnected) {
            _channel?.sink.close();
          } else {
            _connectToServer();
          }
        },
        child: Icon(_isConnected ? Icons.stop : Icons.play_arrow),
      ),
    );
  }

  @override
  void dispose() {
    _channel?.sink.close();
    super.dispose();
  }
}
```

### 3. Sử dụng trong main.dart:
```dart
import 'package:flutter/material.dart';

void main() {
  runApp(MyApp());
}

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Video Stream App',
      home: VideoStreamWidget(),
    );
  }
}
```

## Ghi chú:
- Thay `YOUR_SERVER_IP` bằng IP thực của máy chạy server
- Nếu test trên emulator, có thể dùng `10.0.2.2:8081`
- Nếu test trên device thật, dùng IP của máy tính (vd: `192.168.1.100:8081`)
- Server phải chạy trước khi Flutter app kết nối

In [ ]:
# Test kết nối tới server để xem frames
# Chạy cell này trong notebook riêng hoặc terminal riêng

import asyncio
import websockets
import json
import base64
from PIL import Image
import io

async def test_video_stream():
    uri = "ws://localhost:8081"
    frame_count = 0
    
    try:
        print(f"🔗 Connecting to {uri}...")
        
        async with websockets.connect(uri) as websocket:
            print("✅ Connected to video server!")
            
            async for message in websocket:
                try:
                    data = json.loads(message)
                    
                    if data["type"] == "video_info":
                        print(f"📹 Video Info:")
                        print(f"   📏 Resolution: {data['width']}x{data['height']}")
                        print(f"   ⚡ FPS: {data['fps']}")
                        print(f"   🎬 Total Frames: {data['total_frames']}")
                        print("-" * 50)
                        
                    elif data["type"] == "video_frame":
                        frame_count += 1
                        
                        # Chỉ log mỗi 10 frames để tránh spam
                        if frame_count % 10 == 0:
                            print(f"📺 Received frame #{data['frame_id']} (Client: {frame_count})")
                            
                            # Decode và save sample frame
                            if frame_count == 50:  # Save frame thứ 50 làm mẫu
                                image_data = base64.b64decode(data["image_data"])
                                with open("sample_frame.jpg", "wb") as f:
                                    f.write(image_data)
                                print("💾 Saved sample frame as 'sample_frame.jpg'")
                        
                        # Stop after 100 frames for testing
                        if frame_count >= 100:
                            print(f"🛑 Test complete - received {frame_count} frames")
                            break
                            
                except json.JSONDecodeError:
                    print("❌ Invalid JSON received")
                except Exception as e:
                    print(f"❌ Frame processing error: {e}")
                    
    except websockets.exceptions.ConnectionClosed:
        print("🔌 Connection closed")
    except ConnectionRefusedError:
        print("❌ Cannot connect - Server not running on port 8081")
    except Exception as e:
        print(f"❌ Client error: {e}")

# Chạy test
print("🧪 Testing Video Stream Connection")
print("=" * 40)

# Uncomment để chạy test
# await test_video_stream()